## Обработка датасета для генератора

In [1]:
import numpy as np
import pandas as pd

In [2]:
size = 4 * 10**4 # Размер блока данных
readerCSV = pd.read_csv("/Users/evgenii/DDoS Dataset/final_dataset.csv",sep=",",chunksize=size)
ddos_result = []
i = 0
number_of_objects = 12794627
iter_count = number_of_objects/size 

for data in readerCSV:
    data = data.fillna(0)
    cnt_data = data.groupby(['Src IP']).mean()[['Flow Duration', 'Tot Fwd Pkts', 'Tot Bwd Pkts', 'TotLen Fwd Pkts', 
                                               'TotLen Bwd Pkts', 'Fwd IAT Mean' , 'Bwd IAT Mean' , 'Fwd PSH Flags', 
                                               'Bwd PSH Flags', 'Fwd URG Flags', 'Bwd URG Flags', 'Pkt Len Mean', 
                                               'SYN Flag Cnt', 'RST Flag Cnt', 'PSH Flag Cnt', 'ACK Flag Cnt' , 
                                               'URG Flag Cnt', 'CWE Flag Count', 'ECE Flag Cnt', 'Idle Mean', 'Active Mean']]
    
    dst_ip_data = data.groupby(['Src IP']).agg({'Dst IP': set})
    protocol_data = data.groupby(['Src IP']).agg({'Protocol': set})
    dst_port_data = data.groupby(['Src IP']).agg({'Dst Port': set})
    src_ports_data = data.groupby(['Src IP']).agg({'Src Port': set})
    labels_data = data.groupby(['Src IP']).agg({'Label': set})
    ddos_data = src_ports_data.join(dst_ip_data).join(dst_port_data).join(protocol_data).join(labels_data).join(cnt_data)
    ddos_result.append(ddos_data)
    
    i += 1
    percent = i / iter_count * 100
    print("Чтение данных: " + "{0:.2f}".format(percent if percent < 100 else 100) + "%", end="\r")
print()
print('finished')
readerCSV = None
data = None

Чтение данных: 100.00%
finished


In [3]:
intermediate_res = pd.concat(ddos_result,sort=False)
ddos_result = []
intermediate_res

,Src Port,Dst IP,Dst Port,Protocol,Label,Flow Duration,Tot Fwd Pkts,Tot Bwd Pkts,TotLen Fwd Pkts,TotLen Bwd Pkts,...,Pkt Len Mean,SYN Flag Cnt,RST Flag Cnt,PSH Flag Cnt,ACK Flag Cnt,URG Flag Cnt,CWE Flag Count,ECE Flag Cnt,Idle Mean,Active Mean
Src IP,,,,,,,,,,,,,,,,,,,,,
192.168.1.101,"{4100, 2055, 2057, 2061, 2068, 2069, 2076, 208...","{97.74.144.108, 97.74.104.201, 74.55.1.4}","{80, 443}",{6},{ddos},1.563599e+07,6.884336,8.625909,186.352280,7788.161269,...,207.223648,0.458030,0.0,0.002644,0.541970,0.0,0.0,0.0,6.644611e+06,8.336376e+04
192.168.1.102,"{3076, 3077, 3079, 3082, 3083, 3088, 3090, 309...","{69.84.133.138, 97.74.104.201, 74.55.1.4}",{80},{6},{ddos},1.664876e+07,84.918768,159.750700,293.344538,216019.442577,...,385.028472,0.521008,0.0,0.005602,0.478992,0.0,0.0,0.0,1.003268e+06,3.734518e+05
192.168.1.103,"{2049, 2563, 2052, 2053, 2054, 3075, 3258, 358...","{69.84.133.138, 97.74.144.108, 74.55.1.4, 208....",{80},{6},{ddos},2.002657e+07,38.111709,62.118439,206.213997,83564.204576,...,318.998784,0.410498,0.0,0.008075,0.589502,0.0,0.0,0.0,1.711862e+06,5.116172e+05
192.168.1.104,"{21574, 18449, 18450, 18451, 20523, 20524, 205...","{67.220.214.50, 203.73.24.75, 97.74.144.108, 7...","{80, 443}",{6},{ddos},6.761965e+06,5.443097,9.551819,65.679312,10077.454830,...,179.230691,0.474775,0.0,0.000000,0.525225,0.0,0.0,0.0,3.850293e+05,1.631446e+04
192.168.1.105,"{17408, 18560, 17412, 17418, 18582, 18585, 184...",{97.74.104.201},"{80, 443}",{6},{ddos},5.341630e+07,71.464286,124.642857,350.821429,166482.154762,...,504.460329,0.654762,0.0,0.023810,0.345238,0.0,0.0,0.0,7.704307e+06,3.369959e+06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96.6.49.94,{443},"{172.31.65.113, 172.31.65.65, 172.31.64.69}","{51987, 49972, 50030}",{6},{Benign},1.890667e+03,3.000000,1.000000,31.000000,0.000000,...,6.200000,0.000000,0.0,0.000000,1.000000,0.0,0.0,0.0,0.000000e+00,0.000000e+00
96.6.55.102,{443},{172.31.69.4},{49802},{6},{Benign},1.000000e+00,2.000000,0.000000,31.000000,0.000000,...,20.666667,1.000000,0.0,0.000000,1.000000,0.0,0.0,0.0,0.000000e+00,0.000000e+00
97.68.80.146,{37767},{172.31.65.76},{445},{6},{Benign},4.814400e+04,3.000000,1.000000,0.000000,0.000000,...,0.000000,0.000000,0.0,1.000000,0.000000,0.0,0.0,0.0,0.000000e+00,0.000000e+00


In [4]:
cnt_data = intermediate_res.groupby(['Src IP']).mean()[['Flow Duration', 'Tot Fwd Pkts', 'Tot Bwd Pkts', 'TotLen Fwd Pkts', 
                                           'TotLen Bwd Pkts', 'Fwd IAT Mean' , 'Bwd IAT Mean' , 'Fwd PSH Flags', 
                                           'Bwd PSH Flags', 'Fwd URG Flags', 'Bwd URG Flags', 'Pkt Len Mean', 
                                           'SYN Flag Cnt', 'RST Flag Cnt', 'PSH Flag Cnt', 'ACK Flag Cnt' , 
                                           'URG Flag Cnt', 'CWE Flag Count', 'ECE Flag Cnt', 'Idle Mean', 'Active Mean']]

dst_ip_data = intermediate_res.groupby(['Src IP'])['Dst IP'].apply(np.hstack).to_frame()
protocol_data = intermediate_res.groupby(['Src IP'])['Protocol'].apply(np.hstack).to_frame()
dst_port_data = intermediate_res.groupby(['Src IP'])['Dst Port'].apply(np.hstack).to_frame()
src_ports_data = intermediate_res.groupby(['Src IP'])['Src Port'].apply(np.hstack).to_frame()
labels_data = intermediate_res.groupby(['Src IP'])['Label'].apply(np.hstack).to_frame()
result_data = dst_ip_data.join(protocol_data).join(dst_port_data).join(src_ports_data).join(labels_data).join(cnt_data)
result_data

,Dst IP,Protocol,Dst Port,Src Port,Label,Flow Duration,Tot Fwd Pkts,Tot Bwd Pkts,TotLen Fwd Pkts,TotLen Bwd Pkts,...,Pkt Len Mean,SYN Flag Cnt,RST Flag Cnt,PSH Flag Cnt,ACK Flag Cnt,URG Flag Cnt,CWE Flag Count,ECE Flag Cnt,Idle Mean,Active Mean
Src IP,,,,,,,,,,,,,,,,,,,,,
1.0.132.203,[{172.31.64.91}],[{17}],[{38840}],[{60449}],[{Benign}],2.002600e+04,2.000000,0.000000,393.000000,0.000000,...,141.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1.0.176.98,"[{172.31.67.104}, {172.31.67.48}, {172.31.64.4...","[{6}, {6}, {6}, {6}, {6}, {6}, {6}, {6}, {6}, ...","[{445}, {445}, {445}, {445}, {445}, {445}, {44...","[{15281}, {50483}, {52319}, {30317}, {48026}, ...","[{Benign}, {Benign}, {Benign}, {Benign}, {Beni...",6.889161e+05,4.602273,3.079545,225.170455,177.477273,...,35.947596,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1.1.196.153,"[{172.31.64.99}, {172.31.65.48}, {172.31.65.48...","[{6}, {6}, {6}, {6}]","[{445}, {445}, {445}, {445}]","[{63015}, {56757}, {58357}, {63057}]","[{Benign}, {Benign}, {Benign}, {Benign}]",6.709760e+05,4.500000,2.750000,184.250000,188.500000,...,31.817308,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1.1.207.23,"[{172.31.67.19}, {172.31.67.19}, {172.31.64.14...","[{6}, {6}, {6}, {6}, {6}, {6}, {6}]","[{445}, {445}, {445}, {445}, {445}, {445}, {445}]","[{63898}, {63869}, {56816}, {56785}, {54095}, ...","[{Benign}, {Benign}, {Benign}, {Benign}, {Beni...",5.310179e+05,3.857143,2.285714,135.000000,73.142857,...,20.814286,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1.1.223.186,"[{172.31.67.62}, {172.31.67.62}, {172.31.67.62}]","[{6}, {6}, {6}]","[{445}, {445}, {445}]","[{26367}, {26340}, {26290}]","[{Benign}, {Benign}, {Benign}]",6.478580e+05,4.333333,3.000000,219.666667,114.000000,...,33.366667,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99.229.225.189,"[{172.31.65.25}, {172.31.65.25}]","[{6}, {6}]","[{30303}, {30303}]","[{54058}, {52240}]","[{Benign}, {Benign}]",2.999468e+06,2.000000,0.000000,0.000000,0.000000,...,0.000000,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
99.230.229.253,[{172.31.64.109}],[{6}],[{30303}],[{64634}],[{Benign}],2.160461e+06,3.000000,0.000000,0.000000,0.000000,...,0.000000,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
99.26.255.249,[{172.31.64.113}],[{6}],[{3389}],[{54288}],[{Benign}],7.665771e+07,8.000000,17.000000,1042.000000,1766.000000,...,108.000000,0.0,0.0,1.0,0.0,0.0,0.0,0.0,19046115.0,19519369.0


In [5]:
for index, row in result_data.iterrows():
    p = set()
    for d in row['Dst IP']:
        p |= d
  
    result_data.loc[index,'Dst IP'] = " ".join(map(str, p))
    
    p = set()
    for d in row['Protocol']:
        p |= d
    result_data.loc[index,'Protocol'] = " ".join(map(str, p))
    
    p = set()
    for d in row['Dst Port']:
        p |= d
    result_data.loc[index,'Dst Port'] = " ".join(map(str, p))
    
    p = set()
    for d in row['Src Port']:
        p |= d
    result_data.loc[index,'Src Port'] = " ".join(map(str, p))
    
    p = set()
    for d in row['Label']:
        p |= d
    result_data.loc[index,'Label'] = " ".join(map(str, p))

result_data

,Dst IP,Protocol,Dst Port,Src Port,Label,Flow Duration,Tot Fwd Pkts,Tot Bwd Pkts,TotLen Fwd Pkts,TotLen Bwd Pkts,...,Pkt Len Mean,SYN Flag Cnt,RST Flag Cnt,PSH Flag Cnt,ACK Flag Cnt,URG Flag Cnt,CWE Flag Count,ECE Flag Cnt,Idle Mean,Active Mean
Src IP,,,,,,,,,,,,,,,,,,,,,
1.0.132.203,172.31.64.91,17,38840,60449,Benign,2.002600e+04,2.000000,0.000000,393.000000,0.000000,...,141.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1.0.176.98,172.31.64.45 172.31.65.10 172.31.64.80 172.31....,6,445,52097 24706 31749 23054 19727 18706 13588 5838...,Benign,6.889161e+05,4.602273,3.079545,225.170455,177.477273,...,35.947596,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1.1.196.153,172.31.64.99 172.31.65.48,6,445,63057 58357 56757 63015,Benign,6.709760e+05,4.500000,2.750000,184.250000,188.500000,...,31.817308,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1.1.207.23,172.31.67.19 172.31.64.14,6,445,56816 56785 64357 57734 63898 63869 54095,Benign,5.310179e+05,3.857143,2.285714,135.000000,73.142857,...,20.814286,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1.1.223.186,172.31.67.62,6,445,26290 26340 26367,Benign,6.478580e+05,4.333333,3.000000,219.666667,114.000000,...,33.366667,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99.229.225.189,172.31.65.25,6,30303,52240 54058,Benign,2.999468e+06,2.000000,0.000000,0.000000,0.000000,...,0.000000,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
99.230.229.253,172.31.64.109,6,30303,64634,Benign,2.160461e+06,3.000000,0.000000,0.000000,0.000000,...,0.000000,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
99.26.255.249,172.31.64.113,6,3389,54288,Benign,7.665771e+07,8.000000,17.000000,1042.000000,1766.000000,...,108.000000,0.0,0.0,1.0,0.0,0.0,0.0,0.0,19046115.0,19519369.0


In [6]:
ddos_data = result_data[result_data['Label'] == 'ddos']
ddos_data.to_csv("ddos_src.csv")
len(ddos_data)

34

In [7]:
benign_data = result_data[result_data['Label'] == 'Benign']
len(benign_data)

36724